In [326]:
import requests
from urllib.parse import urlparse
import json
import re
import os
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm

In [328]:
url = 'https://books.toscrape.com/'
user = UserAgent()
headers = {
    "User-Agent":user.random,
}
params = {}
session = requests.session()
response = session.get(url=url, params=params, headers=headers)

In [329]:
def get_categories_from_response(response):
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, features='html.parser')
    else:
        print(response.status_code)
    url_categories = []
    result = soup.find_all(name="a", attrs={"href":re.compile("^catalogue/category/books/")})
    for i in result:
        url_categories.append(i.get("href"))
    url_categories_links = []
    for i in url_categories:
        url_categories_links.append(f'{url}{i}')
    return url_categories_links

In [330]:
url_categories_links = get_categories_from_response(response)

In [331]:
def get_url_from_row(url):
    result = []
    response = session.get(url=url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, features='html.parser')
    data_page_article = soup.find(name="ol", attrs={"class":"row"}).find_all(name="h3")
    for card in data_page_article:
        result.append("/".join(url.split("/")[:4]+(card.find("a").get('href')).split("/")[-2:]))
    return result

In [332]:
def urls_from_paginator(urls):
        url_from_paginator = []
        for url in tqdm(urls):
                while url != None:
                        response = session.get(url=url, params=params, headers=headers)
                        soup = BeautifulSoup(response.text, features='html.parser')
                        if soup.find(name="li", attrs={"class":"next"}) is not None:
                                next_lin="/".join(url.split("/")[:-1]+[soup.find(name="li", attrs={"class":"next"}).findChild("a").get("href")])
                                url_from_paginator.append(next_lin)
                                url = next_lin
                        else:
                                url = None
        result = url_from_paginator
        return result

In [333]:
def all_books_links(urls):
    finall_all_links = []
    for url in tqdm(url_categories_links):
        link_from_row = get_url_from_row(url)
        finall_all_links.extend(link_from_row)
        link_from_paginator = url_from_paginator(url)
        if link_from_paginator:
            for link_paginator in link_from_paginator:
                link_from_row_pag = get_url_from_row(link_paginator)
                finall_all_links.extend(link_from_row_pag)
    return finall_all_links


In [334]:
boks_links = all_books_links(url_categories_links)

100%|██████████| 50/50 [01:02<00:00,  1.25s/it]


In [335]:
columns = ['name','price', 'count', 'description']
result_data = pd.DataFrame(columns = columns)

In [339]:
def data_extraxt(url):
    result = {}
    response = session.get(url=url, params=params, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, features='html.parser')
    else:
        print(response.status_code)

    card = soup.find(name="div", attrs={"class":"product_main"})
    if card:
        result['name'] = card.find(name="h1").text
        price = card.find(name="p", attrs={"class":"price_color"}).text
        result['price'] = float(price[2:])
        count = card.find(name="p", attrs={"class":"instock availability"}).text
        pattern = r'(\d{1,9})'
        count = int(re.search(pattern,count).group())
        result['count'] = count
    descriptions = soup.find(name="article", attrs={"class":"product_page"}).find(name="p", attrs={'class': False, 'id': False})
    if descriptions:
        result['description'] = descriptions.text
    return result
    
    
    

In [341]:
for index, link in enumerate(tqdm(boks_links)):
    data_page = data_extraxt(link)
    result_data.loc[index] = data_page.get('name'), data_page.get('price'), data_page.get('count'), data_page.get('description')
   

100%|██████████| 1117/1117 [03:54<00:00,  4.76it/s]


In [343]:
result_data.head()

,name,price,count,description
0,It's Only the Himalayas,45.17,19,"âWherever you go, whatever you do, just . . ..."
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,15,Acclaimed travel writer Rick Antonson sets his...
2,See America: A Celebration of Our National Par...,48.87,14,To coincide with the 2016 centennial anniversa...
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,8,With a new foreword by Tim Ferriss â¢Thereâ...
4,Under the Tuscan Sun,37.33,7,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...


In [344]:
result_data.to_json('file.json', orient = 'split', compression = 'infer', index = 'true')